In [1]:
import numpy as np
import pandas as pd

from keras.models import Sequential, Model
from keras.layers import Input, Embedding
from keras.layers import add
from keras.regularizers import l2, l1
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.layers.convolutional import *

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
np.random.seed(12345)

### Download and extract MovieLens 1M data file

In [2]:
! wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"

--2018-07-15 09:52:15--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.34.235
Connecting to files.grouplens.org (files.grouplens.org)|128.101.34.235|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1         100%[===================>]   5.64M  20.0MB/s    in 0.3s    

2018-07-15 09:52:16 (20.0 MB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]



In [3]:
! unzip -o -qq ml-1m.zip

In [4]:
! ls ml-1m

movies.dat  ratings.dat  README  users.dat


In [5]:
data_path = 'ml-1m/'

## Set up data

We're working with the movielens data, which contains one rating per row, like this:

In [7]:
ratings = pd.read_csv(data_path + 'ratings.dat', names = ['userId','movieId','rating','timestamp'] , header=None, delimiter  = '::')
ratings.head()

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [10]:
print("Number of ratings :", len(ratings))

Number of ratings : 1000209


Just for display purposes, let's read in the movie names too.

In [13]:
movie_names = pd.read_csv(data_path + 'movies.dat', names = ['movieId','title','genres'] , header=None,  delimiter  = '::')
movie_dict = movie_names.set_index('movieId')['title'].to_dict()

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [14]:
movie_names.head()

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [18]:
[(k,movie_dict[k]) for k in list(movie_dict)][:10]

[(1, 'Toy Story (1995)'),
 (2, 'Jumanji (1995)'),
 (3, 'Grumpier Old Men (1995)'),
 (4, 'Waiting to Exhale (1995)'),
 (5, 'Father of the Bride Part II (1995)'),
 (6, 'Heat (1995)'),
 (7, 'Sabrina (1995)'),
 (8, 'Tom and Huck (1995)'),
 (9, 'Sudden Death (1995)'),
 (10, 'GoldenEye (1995)')]

In [19]:
# Get distinct Ids
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [20]:
# Build dictionaries
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

We update the movie and user ids so that they are contiguous integers, which we want when using embeddings.

In [21]:
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

In [22]:
user_min, user_max, movie_min, movie_max = (ratings.userId.min(), 
    ratings.userId.max(), ratings.movieId.min(), ratings.movieId.max())
user_min, user_max, movie_min, movie_max

(0, 6039, 0, 3705)

In [26]:
users_count = ratings.userId.nunique()
movies_count = ratings.movieId.nunique()
users_count, movies_count

(6040, 3706)

Let's try with 40 as the number of latent factors in each embedding.

In [45]:
factors_count = 40

Randomly split into training and validation.

In [46]:
mask = np.random.rand(len(ratings)) < 0.8
training_data = ratings[mask]
validation_data = ratings[~mask]

##  Define model based on embeddings

In [47]:
def embedding_input(layer_name, input_size, output_size, reg):
    inp = Input(shape=(1,), dtype='int64', name=layer_name)
    return inp, Embedding(input_size, output_size, input_length=1, embeddings_regularizer=l2(reg))(inp)

In [48]:
user_input, user_embedding = embedding_input('user_input', users_count, factors_count, 1e-4)
movie_input, movie_embedding = embedding_input('movie_input', movies_count, factors_count, 1e-4)

In [59]:
x = add([user_embedding, movie_embedding])
x = Flatten()(x)
x = Dropout(0.1)(x)
x = Dense(100, activation='elu')(x)
x = Dropout(0.3)(x)
x = Dense(20, activation='elu')(x)
x = Dense(1)(x)

model = Model([user_input, movie_input], x)
model.compile(Adam(0.005), loss='mse')

In [60]:
model.fit([training_data.userId, training_data.movieId], training_data.rating, batch_size=20000, epochs=20, 
          validation_data=([validation_data.userId, validation_data.movieId], validation_data.rating))

Train on 800419 samples, validate on 199790 samples
Epoch 1/20
800419/800419 [==============================] - 1s 2us/step - loss: 2.9415 - val_loss: 0.8930
Epoch 2/20
800419/800419 [==============================] - 1s 1us/step - loss: 0.9251 - val_loss: 0.8739
Epoch 3/20
800419/800419 [==============================] - 1s 1us/step - loss: 0.9037 - val_loss: 0.8675
Epoch 4/20
800419/800419 [==============================] - 1s 1us/step - loss: 0.8967 - val_loss: 0.8660
Epoch 5/20
800419/800419 [==============================] - 1s 1us/step - loss: 0.8900 - val_loss: 0.8613
Epoch 6/20
800419/800419 [==============================] - 1s 1us/step - loss: 0.8834 - val_loss: 0.8591
Epoch 7/20
800419/800419 [==============================] - 1s 1us/step - loss: 0.8798 - val_loss: 0.8549
Epoch 8/20
800419/800419 [==============================] - 1s 1us/step - loss: 0.8719 - val_loss: 0.8487
Epoch 9/20
800419/800419 [==============================] - 1s 1us/step - loss: 0.8656 - val_loss: 0

In [61]:
# fine tune with a smaller learning rate
model.compile(Adam(0.001), loss='mse') 

In [62]:
model.fit([training_data.userId, training_data.movieId], training_data.rating, batch_size=20000, epochs=100, 
          validation_data=([validation_data.userId, validation_data.movieId], validation_data.rating))

Train on 800419 samples, validate on 199790 samples
Epoch 1/100
800419/800419 [==============================] - 1s 2us/step - loss: 0.8277 - val_loss: 0.8239
Epoch 2/100
800419/800419 [==============================] - 1s 1us/step - loss: 0.8162 - val_loss: 0.8177
Epoch 3/100
800419/800419 [==============================] - 1s 1us/step - loss: 0.8112 - val_loss: 0.8146
Epoch 4/100
800419/800419 [==============================] - 1s 1us/step - loss: 0.8079 - val_loss: 0.8120
Epoch 5/100
800419/800419 [==============================] - 1s 1us/step - loss: 0.8054 - val_loss: 0.8092
Epoch 6/100
800419/800419 [==============================] - 1s 1us/step - loss: 0.8027 - val_loss: 0.8074
Epoch 7/100
800419/800419 [==============================] - 1s 1us/step - loss: 0.8001 - val_loss: 0.8054
Epoch 8/100
800419/800419 [==============================] - 1s 1us/step - loss: 0.7976 - val_loss: 0.8033
Epoch 9/100
800419/800419 [==============================] - 1s 1us/step - loss: 0.7937 - va

800419/800419 [==============================] - 1s 1us/step - loss: 0.7295 - val_loss: 0.7721
Epoch 77/100
800419/800419 [==============================] - 1s 1us/step - loss: 0.7283 - val_loss: 0.7710
Epoch 78/100
800419/800419 [==============================] - 1s 1us/step - loss: 0.7277 - val_loss: 0.7717
Epoch 79/100
800419/800419 [==============================] - 1s 1us/step - loss: 0.7272 - val_loss: 0.7722
Epoch 80/100
800419/800419 [==============================] - 1s 1us/step - loss: 0.7275 - val_loss: 0.7706
Epoch 81/100
800419/800419 [==============================] - 1s 1us/step - loss: 0.7264 - val_loss: 0.7712
Epoch 82/100
800419/800419 [==============================] - 1s 1us/step - loss: 0.7263 - val_loss: 0.7700
Epoch 83/100
800419/800419 [==============================] - 1s 1us/step - loss: 0.7247 - val_loss: 0.7718
Epoch 84/100
800419/800419 [==============================] - 1s 1us/step - loss: 0.7240 - val_loss: 0.7695
Epoch 85/100
800419/800419 [=============

In [63]:
model.save('Sample6.h5')

We can use the model to generate predictions by passing a pair of ints - a user id and a movie id. For instance, this predicts that user #3 would really enjoy movie #6.

In [54]:
model.predict([np.array([3]), np.array([6])])

array([[4.2533917]], dtype=float32)

Let's export the user and movie dictionaries we have as we will need them in the prediction .NET sample

In [55]:
import csv
def saveDictionary(dict, fileName) :
    f = open(fileName, "w", newline='')
    w = csv.writer(f)
    for key, val in dict.items():
        w.writerow([key, val])
    f.close()

In [64]:
saveDictionary(movie_dict, "movies.csv")
saveDictionary(userid2idx, "userid2index.csv")
saveDictionary(movieid2idx, "movieid2index.csv")